# Pytorch — фреймворк для обучения нейронных сетей и не только

## Зачем нужны библиотеки для обучения нейронных сетей?

* иметь качественные и эффективные реализации лоссов, слоев, оптимизаторов
* стандартизация кода, использование сторонних библиотек/моделей/имплементаций статей от коммьюнити
* ускорение с помощью GPU без написания специального кода

## Как обучаюся нейронные сети?

- методы оптимизации, использующие градиент

- для оптимизации вычисления градиента используют метод обратного распространения ошибки

Поэтому в основе любого фреймворка лежит автоматическое дифференцирование

## Фреймворки

Динамика популярности по поисковым запросам в Google:

![](images/frameworks.png)

Tensorflow и Pytorch — два основных фреймворка на текущий момент. 

## В чём разница?

- Все фреймворки содержат приблизительно одинаковый функционал: модули для построения сетей, оптимизаторы для настройки, полезные утили, ...

- Tensorflow 1 использует статическое определение графа (сначала указанный граф компилируется, а потом через него можно прогонять данные).

- Pytorch и новый Tensorflow 2 используют динамическое определение графа — не нужно строить граф заранее.

## На практике

- Динамическое построение графа — привычная работа в python-парадигме и удобная отладка на ходу.

- На практике стоит немного понимать все популярные фреймворки — приходится изучать и использовать открытые реализации по свежим статьям.

## Pytorch

### Tensors

Тензор — многомерный массив и основной объект в Pytorch. Операции с тензорами происходят почти также, как и с массивами в numpy

In [ ]:
%pylab inline
import torch

In [ ]:
x = torch.Tensor()

print(x, x.type())
print("Tensor's device: ", x.device)

# x.to("cuda")  # выдаст ошибку на машине без настроенной CUDA

In [ ]:
# Создание тензора "из данных"

# np.array([10., 20., 30.])

x = torch.tensor([10., 20., 30.])  
print(f"I'm {x}, my type is {x.type()}")

In [ ]:
# Дополнительно мы можем указать тип тензора с помощью dtype

# np.array([10., 20., 30.], dtype=np.int32)

x = torch.tensor([10., 20., 30.], dtype=torch.long)  
print(f"I'm {x}, my type is {x.type()}")

In [ ]:
# Мы можем создать тензор из np.ndarray

numpy_arr = np.eye(4)
x = torch.from_numpy(numpy_arr)
print(f"I'm {x}, my type is {x.type()}.")

In [ ]:
# Метод from_numpy не создает тензор, он использает тот же участок памяти, что и массив. 
# Поэтому изменив массив -- мы изменим тензор.

numpy_arr[:, 0] = 50
print(x)

In [ ]:
# Можно создавать тензоры и без данных. 

# Ниже несколько примеров, которые почти эквивалентны соответствующим в numpy
x = torch.rand(3,3) # np.random.rand(3,3)
print(f"Random tensor {x}")
x = torch.eye(3) # np.eye(3)
print(f"Identity tensor {x}")
x = torch.ones(4, 5) # np.ones((4,5))
print(f"All-ones tensor {x}")
x = torch.zeros(4, 5) # np.zeros((4,5))
print(f"All-zeros tensor {x}")

In [ ]:
# Pytorch умеет превращать тензор в numpy ndarray

x = torch.rand(2, 2)
x_np = x.numpy()
print(f"I'm {x_np}, my type is {type(x_np)} ")

In [ ]:
# Размер тензора можно узнать с помощью .size(), .shape

x = torch.rand(2, 2)
print("x.shape: ", x.shape, "\nx.size(): ", x.size())

In [ ]:
# Менять размер тензора можно с помощью .view([s_1, s_2, s_3, ..., s_n]).
# Произведение  s_1 * ... * s_n -- должно быть равно количеству элементов.
# Одну из s_i можно заменить на -1, тогда она рассчитается автоматически

x = torch.arange(12)
print(x.view([2, 6]).numpy())
print(x.view([3, -1]).numpy()) 
print(x.view([2, 3, -1]).numpy())
try:
    print(x.view([2, 5]).numpy())
except RuntimeError as e:
    print(f"Wrong dimentions produce the following error: {e}")

In [ ]:
# В заключении этого блока покажем, что изменять значение тензора можно прямым обращением по индексу.

x = torch.arange(25).view((-1, 5))
print(x)
print(x[2,4])
print(x[2])
print(x[[0,1,2,3], 0])
x[[0,1,2,3,4], 0] = 100
print(x)

### Операции над тензорами

In [ ]:
# Операции над тензорами производятся интуитивно понятно.
# Разберем это на примере двуслойной сети и подсчета 
# лосса -- среднеквадратичного отклонения

batch_size = 3
n_features = 10
H_n = 30

# сначала создадим все необходимые тензоры -- рандомный вход, выход и веса. Это делать мы уже умеем )

y_real = torch.rand(batch_size)
x = torch.rand(batch_size, n_features)

w_1 = torch.rand(n_features, H_n)
b_1 = torch.rand(H_n)

w_2 = torch.rand(H_n, 1)
b_2 = torch.rand(1)

y_h_1 = (x.matmul(w_1) + b_1).clamp(min=0)  # relu
y_pred = y_h_1.matmul(w_2)+b_2
print('Predictions:')
print(y_pred)

loss = (y_pred - y_real).pow(2).sum()
print('Loss:')
print(loss)

## Автодифференцирование

Разберёмся, как выглядит автодифференцирование на практике

In [ ]:
a = torch.tensor(2.)
b = torch.tensor(1.)

c = a + b
d = b + 1
e = c * d

print(a, b, c, d, e)

Мы можем представить это в виде следующего графа:
<img src="images/tree-eval.png" width="600">

Тепрь посчитаем частные производные по $a$ и $b$.
$$\frac{\partial e}{\partial a} = \frac{\partial e}{\partial c} \frac{\partial c}{\partial a}$$
$$\frac{\partial e}{\partial b} = \frac{\partial e}{\partial c} \frac{\partial c}{\partial b} + 
\frac{\partial e}{\partial d} \frac{\partial d}{\partial b}$$

Можно заметить, что, считая поизводные мы спускаемся про графу сверху вниз, считая производные одного узла по соседнему.

<img src="images/tree-eval-derivs.png" width="600">

- $\frac{\partial e}{\partial a}$ равна произведению значений на ребрах по пути из $a$ в $e$.
- Из $b$ в $e$ идет два пути, и в формуле выше мы видим что мы суммируем значения, полученные для каждого из путей.

Подробнее про бэкпроп: http://colah.github.io/posts/2015-08-Backprop/

Как автоград устроен в Pytorch: https://youtu.be/MswxJw-8PvE




Чтобы посчитать градиент в Pytorch, нужно обратиться к полю .grad

In [ ]:
a = torch.tensor(2., requires_grad=True)
b = torch.tensor(1., requires_grad=True)

c = a + b
d = b + 1
e = c * d
e.backward()

print("de/da = ", a.grad.item(), "de/db = ", b.grad.item())

In [ ]:
c.backward()

print("dc/da = ", a.grad.item(), "dc/db = ", b.grad.item())

# Почему это неправильно?

In [ ]:
# zero gradients !

a.grad.data.zero_()
b.grad.data.zero_()
c.backward()

print("dc/da = ", a.grad.item(), "dc/db = ", b.grad.item())

In [ ]:
# Теперь напишем бекпроп и зафитим случайную выборку )
# источник: https://github.com/jcjohnson/pytorch-examples

# Лучше сразу задавать переменную device, чтобы не переписывать потом
device = torch.device('cpu')

# N размер бача; D_in размер кода;
# H размер скрытого слоя; D_out размер выхода
N, D_in, H, D_out = 64, 1000, 100, 10

# Создадим случайную выборку
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# Инициализируем веса сети
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

In [ ]:
learning_rate = 1e-6

for t in range(100):
    
    # Прямой проход
    y_pred = (x.mm(w1).clamp(min=0)).mm(w2)
    
    # Считаем средне-квадратичное отклонение -- лосс
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    # Считаем производные по параметрам 
    loss.backward()
    
    with torch.no_grad():
        
        # Градиентный спуск
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Зануляем градиенты
        w1.grad.zero_()
        w2.grad.zero_()

## torch.nn, torch.optim

Удобство библиотеки не ограничивается одним только автодифференцированием. Многое из того что мы написали выше уже реализовано.

В частности, стандартные слои, функции активаций, лоссы, оптимизаторы. Вот как выгдядит упрощенная версия кода, использующая модуль torch.nn и torch.optim

In [ ]:
# В моделе класса Sequential операции выполняются одна за другой. 
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),  # мы не определяем параметры сами -- за это отвечает модель
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

for name, parameter in model.named_parameters():
    print(name, parameter.shape)

In [ ]:
loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters())

for t in range(100):
    
    # Прямой проход
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    optimizer.zero_grad()

    # Обратный проход
    loss.backward()

    # Шаг оптимизации
    optimizer.step()

## nn.Module, nn.Dataset

<img src="images/fashion-mnist-sprite.png" width="600">
</br>

Теперь попробуем обучить классификатор одежды для датасета FashionMNIST (https://github.com/zalandoresearch/fashion-mnist ). 

Основные новые концепции, которые мы усвоим на этом примере — **Dataset,  Dataloader, Model**. Они необходимы для удобной работы с библиотекой.

In [ ]:
from torch.utils.data import DataLoader, Dataset
from utils import download_mnist, save_mnist, load
from pathlib import Path

import torchvision
import torchvision.transforms as transforms

In [ ]:
# Начнем с класса Dataset. По названию можно догадаться, что он нужен для работы с датасетами
# Это очень удобная абстракция, которая в дальнейшем может быть использована в комбинации с 
# Dataloader, для параллельной загрузки данных. Но об этом позже.

# Кастомный датасет должен переопределить два метода: __len__ и __getitem__. 

class FashionMNISTDataset(Dataset):
    def __init__(self, transforms=None, training=True):
        if not Path("mnist.pkl").exists():
            download_mnist()
            save_mnist()
        train_imgs, train_labels, test_imgs, test_labels = load()
        self.training = training
        self.transforms = transforms
        
        if self.training:
            self.imgs = train_imgs
            self.labels = train_labels
        else:
            self.imgs = test_imgs
            self.labels = test_labels
            
    def __getitem__(self, idx):
        img = self.imgs[idx].reshape((-1, 28, 1))
        if self.transforms is not None:
            img = self.transforms(img)
        return img, torch.tensor(self.labels[idx], dtype=torch.long)
    
    def __len__(self):
        return len(self.imgs)

In [ ]:
classes = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
    'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
data_transformation = transforms.Compose([transforms.ToTensor()])
datasets = {
    x: FashionMNISTDataset(
        training=(x == "train"), 
        transforms=data_transformation) for x in ["train", "test"]}

In [ ]:
image, label = datasets["train"][1]
print(image.shape, label)
plt.title(classes[label.numpy()])
plt.imshow(image.numpy().squeeze(), cmap="gray")

In [ ]:
# DataLoader это python iterable, который возвращает элементы Dataset бачами

dataloaders = {
    x: DataLoader(datasets[x], batch_size=10, shuffle=True, num_workers=0)
    for x in ["train", "test"]}
batch = next(iter(dataloaders["train"]))
images, labels = batch
print(images.shape)

In [ ]:
grid = torchvision.utils.make_grid(images, nrow=10)

plt.figure(figsize=(15,15))
plt.imshow(np.transpose(grid, (1,2,0)))

print('labels:', labels)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Модели в питорче наследуются от nn.Module
# В стандартном случае нужно определить только __init__ для описания
# состовляющих частей архитектуры и forward для определения того, 
# как они должны взаимодействовать при прямом проходе

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * 32, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
device = "cpu"
model = Network().to(device)
print(model)
print("\nModel parameters shapes:")
for name, parameter in model.named_parameters():
    print(name, parameter.shape)

In [ ]:
def train(model, dataloader, n_epoch):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(1, n_epoch + 1):
        for batch_id, (image, label) in enumerate(dataloader):
            label, image = label.to(device), image.to(device)
            output = model(image)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_id % 1000 == 0:
                print('Loss :{:.4f} Epoch[{}/{}]'.format(loss.item(), epoch, n_epoch))
    return model

In [ ]:
LR = 0.01
n_epoch = 5
dataloaders = {
    x: DataLoader(datasets[x], batch_size=1000, shuffle=x == "train")
    for x in ["train", "test"]}

model = train(model, dataloader=dataloaders["train"], n_epoch=n_epoch)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

def test(model, dataloader):
    predictions = []
    targets = []
    model.eval()
    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)
            outputs = model(image)
            predicted = torch.argmax(outputs,dim=1)
            predictions.extend(predicted.cpu().numpy())
            targets.extend(label.cpu().numpy())
            
        
    predictions = np.array(predictions)
    targets = np.array(targets)
    print(f"Test Accuracy of the model on the test images: {accuracy_score(targets, predictions)*100} %")
    return predictions, targets

In [ ]:
predictions, targets = test(model, dataloaders["test"])

In [ ]:
ids = np.where((np.array(predictions) != np.array(targets)))[0]
num = 310
image, label = datasets["test"][ids[num]]
print(image.shape, label)
plt.title(f"Real: {classes[targets[ids[num]]]}. Predicted: {classes[predictions[ids[num]]]}")
plt.imshow(image.numpy().squeeze(), cmap="gray")

In [ ]:
from utils import plot_confusion_matrix

cm = confusion_matrix(targets, predictions)
plot_confusion_matrix(cm, classes, normalize=False)